# In this project I am trying to predict demand of certain product codes given historical transactional


In [1]:
import pandas as pd
import re
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 

# Dataset
 
can be found here : https://archive.ics.uci.edu/ml/datasets/online+retail


In [4]:
data = pd.read_excel('OnlineRetail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [114]:
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [115]:
data.dropna(inplace=True)
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

# Create Product type and Colour Type from description using NLP

In [13]:
colours = ['red','orange', 'yellow','green', 'blue', 'indigo','violet','purple','pink','silver', 'gold', 'beige', 'brown', 'grey', 'gray', 'black', 'white', 'cream']

stop_words = set(stopwords.words('english'))
Product_type = []
Colour_type = []
dataset=data.head(50000)
for row in dataset.iloc[:,2]:
  s=" "
  description = re.sub('[^a-zA-Z]'," ", str(row).lower())
  wordsList = nltk.word_tokenize(description)
  wordsList = [nltk.stem.WordNetLemmatizer().lemmatize(w, 'n') for w in wordsList if not w in stop_words]

  flag=False
  for w in wordsList:
    if w in colours:
      Colour_type.append(w)
      flag=True
      break
  if flag==False:
    Colour_type.append("No")

  tagged = nltk.pos_tag(wordsList)

  for tag in tagged:
    if tag[1]=='NN' :
      s+=tag[0] +  " "
  Product_type.append(s)


In [14]:
dataset['Product_Type']=Product_type
dataset['Colour_type']=Colour_type
dataset.head()

<ipython-input-14-7c138b120e97>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Product_Type']=Product_type
<ipython-input-14-7c138b120e97>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Colour_type']=Colour_type


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Product_Type,Colour_type
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,heart light holder,white
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,metal lantern,white
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,cream heart coat hanger,cream
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,union flag water bottle,No
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,heart,red


In [15]:
# We do not need them
X = dataset.drop(["Description","InvoiceDate"],axis=1)

In [16]:
X.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type
0,536365,85123A,6,2.55,17850.0,United Kingdom,heart light holder,white
1,536365,71053,6,3.39,17850.0,United Kingdom,metal lantern,white
2,536365,84406B,8,2.75,17850.0,United Kingdom,cream heart coat hanger,cream
3,536365,84029G,6,3.39,17850.0,United Kingdom,union flag water bottle,No
4,536365,84029E,6,3.39,17850.0,United Kingdom,heart,red


# Created a new feature name "GMV" -> Price * Quantity

In [19]:
X['GMV'] = X['UnitPrice'] * X['Quantity']
X.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV
0,536365,85123A,6,2.55,17850.0,United Kingdom,heart light holder,white,15.30
1,536365,71053,6,3.39,17850.0,United Kingdom,metal lantern,white,20.34
2,536365,84406B,8,2.75,17850.0,United Kingdom,cream heart coat hanger,cream,22.00
3,536365,84029G,6,3.39,17850.0,United Kingdom,union flag water bottle,No,20.34
4,536365,84029E,6,3.39,17850.0,United Kingdom,heart,red,20.34


In [25]:
X[['InvoiceNo', 'StockCode','CustomerID','Country','Product_Type','Colour_type']] = X[['InvoiceNo', 'StockCode','CustomerID','Country','Product_Type','Colour_type']].astype(str)

In [26]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

for col in ["InvoiceNo", "StockCode", "CustomerID","Country", "Product_Type","Colour_type"]:
  X[col] = label_encoder.fit_transform(X[col])

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 79177
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   InvoiceNo     50000 non-null  int64  
 1   StockCode     50000 non-null  int64  
 2   Quantity      50000 non-null  int64  
 3   UnitPrice     50000 non-null  float64
 4   CustomerID    50000 non-null  int64  
 5   Country       50000 non-null  int64  
 6   Product_Type  50000 non-null  int64  
 7   Colour_type   50000 non-null  int64  
 8   GMV           50000 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 3.8 MB


In [29]:
X = X.astype('category')
X.iloc[:, 2] = X.iloc[:, 2].astype(float)
X.iloc[:, 3] = X.iloc[:, 3].astype(float)
X.iloc[:, 8] = X.iloc[:, 3].astype(float)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 79177
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   InvoiceNo     50000 non-null  category
 1   StockCode     50000 non-null  category
 2   Quantity      50000 non-null  float64 
 3   UnitPrice     50000 non-null  float64 
 4   CustomerID    50000 non-null  category
 5   Country       50000 non-null  category
 6   Product_Type  50000 non-null  category
 7   Colour_type   50000 non-null  category
 8   GMV           50000 non-null  float64 
dtypes: category(6), float64(3)
memory usage: 2.3 MB


In [31]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(X, train_size=0.7, random_state = 123)

# Cluster similar purchases based on feature values

In [33]:
kproto = KPrototypes(n_clusters=3, init='Cao')
kproto.fit_predict(train, categorical=[0,1,4,5,6])
print(kproto.cost_)
labels=kproto.labels_

49759184.77822324


In [34]:
train["Cluster number"]=labels
train

<ipython-input-34-b5f872ad72b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Cluster number"]=labels


,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV,Cluster number
26354,880,276,2.0,2.95,476,25,1760,11,2.95,1
20198,717,704,2.0,2.95,1092,25,1929,0,2.95,1
78843,2430,1291,1.0,4.95,872,25,1239,0,4.95,1
25400,851,517,1.0,3.75,735,25,872,11,3.75,1
19735,687,2308,24.0,0.42,705,25,346,6,0.42,1
...,...,...,...,...,...,...,...,...,...,...
10054,400,736,3.0,3.75,224,25,442,13,3.75,1
23634,807,1564,2.0,9.95,113,25,1507,0,9.95,1
26762,893,695,12.0,1.65,228,25,2066,11,1.65,1
44379,1456,1613,1.0,16.95,507,25,212,11,16.95,1


In [44]:
mergedDf = train.merge(pd.DataFrame(dataset["InvoiceDate"]), left_index=True, right_index=True)
mergedDf

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV,Cluster number,InvoiceDate
26354,880,276,2.0,2.95,476,25,1760,11,2.95,1,2010-12-12 13:37:00
20198,717,704,2.0,2.95,1092,25,1929,0,2.95,1,2010-12-09 12:37:00
78843,2430,1291,1.0,4.95,872,25,1239,0,4.95,1,2011-02-01 14:01:00
25400,851,517,1.0,3.75,735,25,872,11,3.75,1,2010-12-12 10:57:00
19735,687,2308,24.0,0.42,705,25,346,6,0.42,1,2010-12-09 10:04:00
...,...,...,...,...,...,...,...,...,...,...,...
10054,400,736,3.0,3.75,224,25,442,13,3.75,1,2010-12-05 16:24:00
23634,807,1564,2.0,9.95,113,25,1507,0,9.95,1,2010-12-10 11:33:00
26762,893,695,12.0,1.65,228,25,2066,11,1.65,1,2010-12-13 09:29:00
44379,1456,1613,1.0,16.95,507,25,212,11,16.95,1,2011-01-05 12:20:00


In [60]:
mergedDf["InvoiceDate"]= pd.to_datetime(mergedDf["InvoiceDate"])


In [45]:
mergedDf.to_csv('mergedDf.csv')

In [46]:
mergedDf = pd.read_csv('mergedDf.csv')
mergedDf.drop(['Unnamed: 0'], axis =1, inplace=True)
mergedDf.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV,Cluster number,InvoiceDate
0,880,276,2.0,2.95,476,25,1760,11,2.95,1,2010-12-12 13:37:00
1,717,704,2.0,2.95,1092,25,1929,0,2.95,1,2010-12-09 12:37:00
2,2430,1291,1.0,4.95,872,25,1239,0,4.95,1,2011-02-01 14:01:00
3,851,517,1.0,3.75,735,25,872,11,3.75,1,2010-12-12 10:57:00
4,687,2308,24.0,0.42,705,25,346,6,0.42,1,2010-12-09 10:04:00


# Create new numerical columns from date

In [61]:
mergedDf['day'] = mergedDf['InvoiceDate'].dt.day

In [63]:
mergedDf['year'] = mergedDf['InvoiceDate'].dt.year

In [64]:
mergedDf['month'] = mergedDf['InvoiceDate'].dt.month

In [66]:
mergedDf.drop(["InvoiceDate"],axis=1, inplace = True)


In [67]:
mergedDf.to_csv('mergedDf.csv')

# Train | Test Split 

In [68]:
from sklearn.model_selection import train_test_split
train_, val_= train_test_split(mergedDf, train_size = 0.7, random_state = 123)

In [69]:
train_y=train_["Cluster number"]
train_x=train_.drop(['Cluster number'],axis=1,inplace=False)

val_y=val_["Cluster number"]
val_x=val_.drop(['Cluster number'],axis=1,inplace=False)

In [70]:
from sklearn.svm import LinearSVC 

In [116]:
import warnings
warnings.filterwarnings('ignore')
model1 = LinearSVC()
model1.fit(train_x,train_y)

LinearSVC()

In [72]:
pred_y = model1.predict(val_x)

In [73]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y,pred_y)

0.9997142857142857

In [91]:
test_Df = test.merge(pd.DataFrame(dataset["InvoiceDate"]), left_index=True, right_index=True)
test_Df

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV,InvoiceDate
17904,617,991,1.0,8.50,1158,25,2229,13,8.50,2010-12-08 12:46:00
66488,2025,921,3.0,2.95,692,25,1419,0,2.95,2011-01-21 14:17:00
58308,1827,852,48.0,0.29,403,25,999,0,0.29,2011-01-14 14:47:00
7456,318,1440,2.0,2.95,857,25,1702,14,2.95,2010-12-05 10:36:00
47461,1543,275,3.0,0.95,1047,25,347,0,0.95,2011-01-07 10:47:00
...,...,...,...,...,...,...,...,...,...,...
32568,1118,1246,6.0,2.95,677,25,984,0,2.95,2010-12-16 09:46:00
37840,1291,822,1.0,1.95,770,25,68,9,1.95,2010-12-19 15:44:00
45541,1491,1493,240.0,4.25,15,0,277,0,4.25,2011-01-06 11:12:00
58093,1824,1729,64.0,8.50,415,6,528,0,8.50,2011-01-14 14:11:00


In [92]:
test_Df['day'] = test_Df['InvoiceDate'].dt.day
test_Df['month'] = test_Df['InvoiceDate'].dt.month
test_Df['year'] = test_Df['InvoiceDate'].dt.year

In [94]:
test_Df= test_Df.drop(["InvoiceDate"],axis=1)

In [96]:
test_Df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 17904 to 78177
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   InvoiceNo     15000 non-null  category
 1   StockCode     15000 non-null  category
 2   Quantity      15000 non-null  float64 
 3   UnitPrice     15000 non-null  float64 
 4   CustomerID    15000 non-null  category
 5   Country       15000 non-null  category
 6   Product_Type  15000 non-null  category
 7   Colour_type   15000 non-null  category
 8   GMV           15000 non-null  float64 
 9   day           15000 non-null  int64   
 10  month         15000 non-null  int64   
 11  year          15000 non-null  int64   
dtypes: category(6), float64(3), int64(3)
memory usage: 1.3 MB


In [97]:
# Prediction of cluster_number on test data

test_Df['Cluster number'] = model1.predict(test_Df)
test_Df 

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV,day,month,year,Cluster number
17904,617,991,1.0,8.50,1158,25,2229,13,8.50,8,12,2010,1
66488,2025,921,3.0,2.95,692,25,1419,0,2.95,21,1,2011,1
58308,1827,852,48.0,0.29,403,25,999,0,0.29,14,1,2011,1
7456,318,1440,2.0,2.95,857,25,1702,14,2.95,5,12,2010,1
47461,1543,275,3.0,0.95,1047,25,347,0,0.95,7,1,2011,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,1118,1246,6.0,2.95,677,25,984,0,2.95,16,12,2010,1
37840,1291,822,1.0,1.95,770,25,68,9,1.95,19,12,2010,1
45541,1491,1493,240.0,4.25,15,0,277,0,4.25,6,1,2011,1
58093,1824,1729,64.0,8.50,415,6,528,0,8.50,14,1,2011,1


# Predict Quantity demand

In [98]:
train_y=train_["Quantity"].astype('int')
train_x=train_.drop(['Quantity'], axis=1,inplace=False)

test_Df_y=test_Df["Quantity"].astype('int')
test_Df_x=test_Df.drop(['Quantity'],axis=1,inplace=False)

Random Forest Classifier

In [99]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(bootstrap=True,ccp_alpha=0.0,
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                                                   n_jobs=None,)
clf.fit(train_x, train_y)

RandomForestClassifier()

In [100]:
prediction_test = clf.predict(test_Df_x)
prediction_test

array([ 1,  2, 24, ...,  4,  2, 12])

In [101]:
from sklearn.metrics import f1_score
f1_score(test_Df_y, prediction_test, average='micro')

0.44

In [102]:
accuracy_score(test_Df_y, prediction_test)

0.44

In [103]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_x, train_y)
knn=neigh.predict(test_Df_x)
print(accuracy_score(test_Df_y, knn))

0.25233333333333335


In [111]:
# Naive base Classifier

from sklearn.naive_bayes import GaussianNB

lr = GaussianNB()
lr.fit(train_x, train_y)
lrc=lr.predict(test_Df_x)
print(accuracy_score(test_Df_y, lrc))

0.0524


In [112]:
# Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier 

dtree_model = DecisionTreeClassifier().fit(train_x, train_y)
dtree_predictions = dtree_model.predict(test_Df_x)
accuracy_score(test_Df_y, dtree_predictions)

0.31553333333333333

In [119]:
test_Df_x

,InvoiceNo,StockCode,UnitPrice,CustomerID,Country,Product_Type,Colour_type,GMV,day,month,year,Cluster number
17904,617,991,8.50,1158,25,2229,13,8.50,8,12,2010,1
66488,2025,921,2.95,692,25,1419,0,2.95,21,1,2011,1
58308,1827,852,0.29,403,25,999,0,0.29,14,1,2011,1
7456,318,1440,2.95,857,25,1702,14,2.95,5,12,2010,1
47461,1543,275,0.95,1047,25,347,0,0.95,7,1,2011,1
...,...,...,...,...,...,...,...,...,...,...,...,...
32568,1118,1246,2.95,677,25,984,0,2.95,16,12,2010,1
37840,1291,822,1.95,770,25,68,9,1.95,19,12,2010,1
45541,1491,1493,4.25,15,0,277,0,4.25,6,1,2011,1
58093,1824,1729,8.50,415,6,528,0,8.50,14,1,2011,1


In [110]:
prediction_test

array([ 1,  2, 24, ...,  4,  2, 12])